In [3]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_corr_triangle
from core.s3 import S3AssetManager


import seaborn as sns
import re, unicodedata
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Optional, List, Tuple, Dict

In [2]:
notebook_name = "comayma_microbiologia_adiflow"
s3 = S3AssetManager(notebook_name=notebook_name)

In [4]:


def generalized_grouped_bar_corporate(
    df: pd.DataFrame,
    formula_col: str = "Formula",             # eje X (categoría)
    group_col: str = "Grupo",                 # series/grupos de barras
    value_col: str = "Valor",                 # valor
    title: str = "Gráfico de barras agrupado",
    order_formula: Optional[List[str]] = None,
    order_group: Optional[List[str]] = None,  # Opcional: fija el orden de los grupos
    # Colores:
    color_discrete_sequence: Optional[List[str]] = None, # Lista de colores para los grupos
    # Estilo:
    font_family: str = "Inter, 'Helvetica Neue', Arial, sans-serif",
    background: str = "#FFFFFF",
    text_fmt: str = ".1f",                    # formato etiqueta de barra
    show_bar_text: bool = True,
    # Títulos y formato de ejes:
    x_axis_title: str = "Categoría",
    y_axis_title: str = "Valor",
    y_tickformat: Optional[str] = None,       # p.ej. ".1%" o ",.1f"
    # Guardado:
    write_html: Optional[str] = None,
    write_image: Optional[str] = None
) -> Tuple[go.Figure, pd.DataFrame]:
    """
    Genera un gráfico de barras agrupadas corporativo generalizado.
    Permite múltiples grupos (más de 2) y no calcula deltas.
    Retorna (fig, df_plot).
    """
    # --- Validaciones ---
    for c in [formula_col, group_col, value_col]:
        if c not in df.columns:
            raise ValueError(f"Columna requerida no encontrada: {c}")

    # --- Agregación ---
    d = (
        df[[formula_col, group_col, value_col]]
        .dropna(subset=[formula_col, group_col, value_col])
        .groupby([formula_col, group_col], as_index=False)[value_col].sum()
    )

    # Orden
    if order_formula is not None:
        d[formula_col] = pd.Categorical(d[formula_col], categories=order_formula, ordered=True)

    # Manejo de orden de grupos
    groups_in_order = order_group
    if groups_in_order is None:
        # Si no se da orden, usar los valores únicos ordenados
        groups_in_order = sorted(d[group_col].unique())
    else:
        # Si se da orden, filtrar por los que existen
        existing_groups = d[group_col].unique()
        groups_in_order = [g for g in order_group if g in existing_groups]

    d[group_col] = pd.Categorical(d[group_col], categories=groups_in_order, ordered=True)

    d = d.sort_values([formula_col, group_col])

    # --- Barras ---
    fig = go.Figure()

    # Mapa de colores
    color_map = {}
    if color_discrete_sequence:
        color_map = {
            group: color_discrete_sequence[i % len(color_discrete_sequence)]
            for i, group in enumerate(groups_in_order)
        }

    for g in groups_in_order:
        dd = d[d[group_col] == g]
        if dd.empty:
            continue

        fig.add_trace(go.Bar(
            name=str(g),
            x=dd[formula_col],
            y=dd[value_col],
            # Asigna color del mapa; si no hay mapa, Plotly usa su ciclo
            marker_color=color_map.get(str(g)),
            marker_line_color="rgba(0,0,0,0.25)",
            marker_line_width=0.6,
            text=dd[value_col] if show_bar_text else None,
            texttemplate=f"%{{text:{text_fmt}}}",
            textposition="outside",
            hovertemplate=(
                f"{group_col}: <b>{g}</b><br>"
                f"{formula_col}: %{{x}}<br>" # <--- CORRECCIÓN AQUÍ
                f"{value_col}: %{{y:{text_fmt}}}<extra></extra>" # <--- CORRECCIÓN AQUÍ (también para y, por consistencia y robustez)
            ),
        ))

    # --- Layout + marco ---
    fig.update_layout(
        title=f"<b>{title}</b>",
        barmode="group",
        bargap=0.25,
        bargroupgap=0.10,
        font=dict(family=font_family, size=12, color="#000000"),
        paper_bgcolor=background,
        plot_bgcolor=background,
        margin=dict(l=40, r=30, t=70, b=90),
        legend=dict(title=group_col, orientation="h", y=1.09, x=0),
        hoverlabel=dict(bgcolor="#FFFFFF", font=dict(color="#000000"), bordercolor="#666666"),
        shapes=[dict(
            type="rect", xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="#000000", width=1.2),
            fillcolor="rgba(0,0,0,0)"
        )]
    )

    # Títulos y formato de ejes
    fig.update_xaxes(
        title_text=x_axis_title,
        showline=True, linecolor="#000000", linewidth=1,
        ticks="outside", tickcolor="#000000",
        tickangle=0, automargin=True
    )
    fig.update_yaxes(
        title_text=y_axis_title,
        showline=True, linecolor="#000000", linewidth=1,
        gridcolor="rgba(0,0,0,0.08)",
        zeroline=True, zerolinecolor="rgba(0,0,0,0.2)",
        automargin=True,
        tickformat=y_tickformat  # e.g., ".1%", ",.1f"
    )

    # Guardado
    if write_html:
        fig.write_html(write_html, include_plotlyjs="cdn")
    if write_image:
        fig.write_image(write_image, width=1400, height=720, scale=2)

    return fig, d

In [7]:
okuo = s3.read_excel("raw/comayma/Resultados Microbiología.xlsx", sheet_name="okuo")
comayma = s3.read_excel("raw/comayma/Resultados Microbiología.xlsx", sheet_name="comayma")
hist = s3.read_excel("raw/comayma/Resultados Microbiología.xlsx", sheet_name="historico")
hist

,Unnamed: 0,Referencia,Coliformes totales (UFC/g),E. Coli (UFC/g),Recuento aeróbico total (UFC/g),Unnamed: 5,Levaduras (UFC/g),Mohos (UFC/g)
0,312.0,NUTRIMAYMA DESARROLLO CERDO PELLET,1200,NaN,4000,NaN,<10,<10
1,312.0,NUTRIMAYMA DESARROLLO CERDO PELLET,660,NaN,9800,NaN,140,60
2,312.0,NUTRIMAYMA DESARROLLO CERDO PELLET,740,NaN,8800,NaN,250,190
3,312.0,NUTRIMAYMA DESARROLLO CERDO PELLET,<10,<10,4200,NaN,80,<10
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,313.0,NUTRIMAYMA FINAL. CERDO PELLET,1800,NaN,3800,NaN,30,30
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,383.0,DESARROLLO CERDO PELLET,300,NaN,4400,NaN,<10,40
8,383.0,DESARROLLO CERDO PELLET,NaN,NaN,360000,NaN,100,100
9,383.0,DESARROLLO CERDO PELLET,NaN,NaN,60000,NaN,100,<10


In [8]:
cols_mohos_lev = ['Levaduras (UFC/g)', 'Mohos (UFC/g)']
for cl in cols_mohos_lev:
    comayma[cl] = pd.to_numeric(comayma[cl], errors="coerce")
    hist[cl] = pd.to_numeric(hist[cl], errors="coerce")


In [9]:
comayma['Mohos y Levaduras (UFC/g)'] = comayma['Levaduras (UFC/g)'] + comayma['Mohos (UFC/g)']
hist['Mohos y Levaduras (UFC/g)'] = hist['Levaduras (UFC/g)'] + hist['Mohos (UFC/g)']
comayma["analisis"] = "comayma"
okuo["analisis"] = "okuo"
hist["analisis"] = "histórico"

cols = ["Coliformes totales (UFC/g)",
        'E. Coli (UFC/g)',
        'Recuento aeróbico total (UFC/g)',
        'Mohos y Levaduras (UFC/g)']
for cl in cols:
    okuo[cl] = pd.to_numeric(okuo[cl], errors="coerce").fillna(1)
    comayma[cl] = pd.to_numeric(comayma[cl], errors="coerce").fillna(1)
    hist[cl] = pd.to_numeric(hist[cl], errors="coerce").fillna(1)

hist_group = hist.groupby(["Referencia","analisis"]).agg(
    coliformes=("Coliformes totales (UFC/g)", "mean"),
    e_coliformes=('E. Coli (UFC/g)', "mean"),
    aerofilos=("Recuento aeróbico total (UFC/g)", "mean"),
    mohos_lev=("Mohos y Levaduras (UFC/g)", "mean"),
).reset_index()
hist_group = hist_group.rename(columns={
    "coliformes": "Coliformes totales (UFC/g)",
    "e_coliformes": "E. Coli (UFC/g)",
    "aerofilos": "Recuento aeróbico total (UFC/g)",
    "mohos_lev": "Mohos y Levaduras (UFC/g)",

})
hist_group

,Referencia,analisis,Coliformes totales (UFC/g),E. Coli (UFC/g),Recuento aeróbico total (UFC/g),Mohos y Levaduras (UFC/g)
0,DESARROLLO CERDO PELLET,histórico,50.833333,1.0,610733.500000,2967.000000
1,FINALIZADOR CERDO PELLET,histórico,347.000000,1.0,6133.333333,133.666667
2,NUTRIMAYMA DESARROLLO CERDO PELLET,histórico,650.250000,1.0,6700.000000,160.500000
3,NUTRIMAYMA FINAL. CERDO PELLET,histórico,1800.000000,1.0,3800.000000,60.000000


In [10]:
okuo_dep = okuo[hist_group.columns]
comayma_dep = comayma[hist_group.columns]

In [11]:

datos = pd.concat([okuo_dep, comayma_dep,hist_group])

datos = datos[datos["Referencia"].notnull()]
datos.groupby(["Referencia","analisis"]).agg(count=("analisis", "count"))
datos.columns

Index(['Referencia', 'analisis', 'Coliformes totales (UFC/g)',
       'E. Coli (UFC/g)', 'Recuento aeróbico total (UFC/g)',
       'Mohos y Levaduras (UFC/g)'],
      dtype='object')

In [12]:
datos

,Referencia,analisis,Coliformes totales (UFC/g),E. Coli (UFC/g),Recuento aeróbico total (UFC/g),Mohos y Levaduras (UFC/g)
0,DESARROLLO CERDO PELLET,okuo,1.000000,1.0,14000.000000,130.000000
1,NUTRIMAYMA DESARROLLO CERDO PELLET,okuo,20.000000,1.0,74000.000000,270.000000
2,FINALIZADOR CERDO PELLET,okuo,1.000000,1.0,12000.000000,20.000000
3,NUTRIMAYMA FINAL. CERDO PELLET,okuo,1.000000,1.0,46000.000000,50.000000
0,FINALIZADOR CERDO PELLET,comayma,1.000000,1.0,5300.000000,1900.000000
1,DESARROLLO CERDO PELLET,comayma,1.000000,1.0,14000.000000,1700.000000
2,NUTRIMAYMA DESARROLLO CERDO PELLET,comayma,1.000000,1.0,7250.000000,310.000000
3,NUTRIMAYMA FINAL. CERDO PELLET,comayma,1.000000,1.0,6100.000000,1500.000000
0,DESARROLLO CERDO PELLET,histórico,50.833333,1.0,610733.500000,2967.000000
1,FINALIZADOR CERDO PELLET,histórico,347.000000,1.0,6133.333333,133.666667


In [13]:
datos["log10_Coliformes_UFCg"] = np.log10(
    datos["Coliformes totales (UFC/g)"]
)
datos["log10_E_Coli_UFCg"] = np.log10(
    datos["E. Coli (UFC/g)"]
)
datos["log10_Mohos_Levaduras_UFCg"] = np.log10(
    datos["Mohos y Levaduras (UFC/g)"]
)
datos["log10_Recuento_Aerofilos_UFCg"] = np.log10(
    datos["Recuento aeróbico total (UFC/g)"]
)

In [14]:

# 2. Especificando una secuencia de colores
colores_corporativos = ["#1C8074", "#c76931", "#666666"]
fig2, df_plot2 = generalized_grouped_bar_corporate(
    df=datos,
    formula_col="Referencia",
    group_col="analisis",
    value_col="log10_Coliformes_UFCg",
    title="log₁₀(Coliformes totales (UFC/g)) por método de medición contrastado con el histórico",
    #order_formula=["Fórmula 4", "Fórmula 1", "Fórmula 2", "Fórmula 3"], # Orden X
    order_group=["okuo", "comayma", "histórico"],          # Orden grupos (y colores)
    color_discrete_sequence=colores_corporativos,
    x_axis_title="Producto",
    y_axis_title="log₁₀(UFC/g)",
   #write_html=f"{ROOT_IMAGEN}/coliformes.html" , # Guarda el archivo en la misma carpeta
    text_fmt=",.2f" # Formato de texto sin decimales
)

print("Gráfico generado. Mostrando el segundo ejemplo:")
fig2.show()
s3.save_plotly_html(fig2, "coliformes.html")

Gráfico generado. Mostrando el segundo ejemplo:


In [15]:

# 2. Especificando una secuencia de colores
colores_corporativos = ["#1C8074", "#c76931", "#666666"]
fig2, df_plot2 = generalized_grouped_bar_corporate(
    df=datos,
    formula_col="Referencia",
    group_col="analisis",
    value_col="log10_E_Coli_UFCg",
    title="log₁₀(E. Coli (UFC/g)) por método de medición contrastado con el histórico",
    #order_formula=["Fórmula 4", "Fórmula 1", "Fórmula 2", "Fórmula 3"], # Orden X
    order_group=["okuo", "comayma", "histórico"],          # Orden grupos (y colores)
    color_discrete_sequence=colores_corporativos,
    x_axis_title="Producto",
    y_axis_title="log₁₀(UFC/g)",
    #write_html=f"{ROOT_IMAGEN}/ecoli.html" , # Guarda el archivo en la misma carpeta
    text_fmt=",.2f" # Formato de texto sin decimales
)

print("Gráfico generado. Mostrando el segundo ejemplo:")
fig2.show()
s3.save_plotly_html(fig2, "ecoli.html")

Gráfico generado. Mostrando el segundo ejemplo:


In [16]:

# 2. Especificando una secuencia de colores
colores_corporativos = ["#1C8074", "#c76931", "#666666"]
fig2, df_plot2 = generalized_grouped_bar_corporate(
    df=datos,
    formula_col="Referencia",
    group_col="analisis",
    value_col="log10_Mohos_Levaduras_UFCg",
    title="log₁₀(Mohos y Levaduras (UFC/g)) por método de medición contrastado con el histórico",
    #order_formula=["Fórmula 4", "Fórmula 1", "Fórmula 2", "Fórmula 3"], # Orden X
    order_group=["okuo", "comayma", "histórico"],          # Orden grupos (y colores)
    color_discrete_sequence=colores_corporativos,
    x_axis_title="Producto",
    y_axis_title="log₁₀(UFC/g)",
    #write_html=f"{ROOT_IMAGEN}/mohos_lev.html" , # Guarda el archivo en la misma carpeta
    text_fmt=",.2f" # Formato de texto sin decimales
)

print("Gráfico generado. Mostrando el segundo ejemplo:")
fig2.show()
s3.save_plotly_html(fig2, "mohos_lev.html")

Gráfico generado. Mostrando el segundo ejemplo:


In [17]:

# 2. Especificando una secuencia de colores
colores_corporativos = ["#1C8074", "#c76931", "#666666"]
fig2, df_plot2 = generalized_grouped_bar_corporate(
    df=datos,
    formula_col="Referencia",
    group_col="analisis",
    value_col="log10_Recuento_Aerofilos_UFCg",
    title="log₁₀(Recuento aeróbico total (UFC/g)) por método de medición contrastado con el histórico",
    #order_formula=["Fórmula 4", "Fórmula 1", "Fórmula 2", "Fórmula 3"], # Orden X
    order_group=["okuo", "comayma", "histórico"],          # Orden grupos (y colores)
    color_discrete_sequence=colores_corporativos,
    x_axis_title="Producto",
    y_axis_title="log₁₀(UFC/g)",
    #write_html=f"{ROOT_IMAGEN}/aerofilos.html" , # Guarda el archivo en la misma carpeta
    text_fmt=",.2f" # Formato de texto sin decimales
)

print("Gráfico generado. Mostrando el segundo ejemplo:")
fig2.show()
s3.save_plotly_html(fig2, f"aerofilos.html")

Gráfico generado. Mostrando el segundo ejemplo:
